In [6]:
# --- Setup: imports & helper for Qiskit >=1.0 ---
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from math import gcd
from fractions import Fraction
from IPython.display import display
from qiskit.visualization import plot_histogram

# Simulator
sim = AerSimulator()

# Helper to run and plot circuits
def run_and_plot(qc, shots=1024):
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()  # no assemble needed
    counts = result.get_counts()
    try:
        display(plot_histogram(counts))
    except Exception:
        print(counts)
    return counts


In [7]:
# --- Imports & setup ---
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from math import gcd
from fractions import Fraction

# Simulator
sim = AerSimulator()

# --- Quantum Fourier Transform (inverse) ---
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

# --- Controlled modular multiplication (specific for N=15) ---
def c_amod15(a, power):
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [11,14]:
            U.swap(0,2)
            U.swap(1,3)
            U.x(range(4))
    U.name = "%i^%i mod 15" % (a, power)
    return U.to_gate().control()

# --- Quantum Phase Estimation circuit for order finding ---
def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    
    # Counting qubits
    for q in range(n_count):
        qc.h(q)
    
    # Auxiliary register in |1>
    qc.x(3+n_count)
    
    # Apply controlled-U operations
    for q in range(n_count):
        qc.append(c_amod15(a, 2**q), [q] + [i+n_count for i in range(4)])
    
    # Apply inverse QFT
    qc.append(qft_dagger(n_count), range(n_count))
    
    # Measure counting qubits
    qc.measure(range(n_count), range(n_count))
    
    return qc

# --- Run Shor for N=15 ---
a = 7  # random co-prime to 15
qc = qpe_amod15(a)

tqc = transpile(qc, sim)
result = sim.run(tqc, shots=1024).result()
counts = result.get_counts()

print("Measurement results:", counts)

# Get most probable phase
phase = max(counts, key=counts.get)
phase_decimal = int(phase, 2)/(2**8)

# Estimate r (order)
r = Fraction(phase_decimal).limit_denominator(15).denominator
print(f"Order r found = {r}")

# Compute factors
factor1 = gcd(a**(r//2)-1, 15)
factor2 = gcd(a**(r//2)+1, 15)
print(f"Factors of 15 are {factor1} and {factor2}")


Measurement results: {'00000000': 520, '10000000': 504}
Order r found = 1
Factors of 15 are 15 and 1
